In [5]:
from google.colab import files 


uploaded = files.upload()


Saving thornhill-Copy1.csv to thornhill-Copy1.csv


In [6]:
from google.colab import files 


uploaded = files.upload()


Saving z1.csv to z1.csv
Saving z2.csv to z2.csv


In [7]:
import pandas as pd
import csv
import numpy as np

In [8]:
#baca file

data = pd.read_csv("thornhill-Copy1.csv")
data2 = pd.read_csv("z1.csv", delimiter=';')
data3 = pd.read_csv("z2.csv", delimiter=';')


In [9]:
#select data
TC1 = np.array(data["TC1"])
TC2 = np.array(data["TC2"])
TC3 = np.array(data["TC3"])
TC4 = np.array(data["TC4"])
TC5 = np.array(data["TC5"])
PC1 =np.array(data["PC1"])
PC2 =np.array(data["PC2"])
PC3 =np.array(data["PC3"])
PC4 =np.array(data["PC4"])
PC5 =np.array(data["PC5"])
LC2 =np.array(data["LC2"])
LC3 =np.array(data["LC3"])
LC4 =np.array(data["LC4"])
LC5 =np.array(data["LC5"])

PV1 = np.array(data2["PV"])
PV2 = np.array(data3["PV"])



In [12]:
def bicoherence(x) :
    LX = 64
    LS = 128
    OV = 0,5

    indices = []
    k1=0
    while True:
        indice = np.array(x[int((LX/2)*k1):int((LX/2)*k1 + LX)])
        indices = np.append(indice,indices)
        #values = indices[k]
        #values = np.asmatrix(values)
        #values = value
        #print(indices)
        k1 = k1+1
        lent= (len(data)-LX/2)
        if len(indices) > lent :
            break

    values = indices.reshape(k1,LX)
    sz1 = values.shape[0]
    numseg = sz1

    x1=np.zeros([LX,LX])
    x2= np.array([])
    o = indices.shape
    w = np.array([])
    Bsp=np.zeros((LX,LX))
    de1=np.zeros((LX,LX))
    de2=np.zeros((LX,LX))
    D1 = np.array([])
    D2 = np.array([])
    Bs = np.array([])
    for ix in range(0,numseg) :
        mui= np.mean(values[ix])
        a= values[ix]-mui
        x1 = a
    #hanning window
        for k in range(0,len(x1)):
            w1= 0.5 - 0.5*np.cos(2*np.pi*(k-1)/(LX-1))
            w = np.append(w,w1)
        for k in range (0,len(x1)):
            abc = w[k]*x1[k]
            x2 = np.append(x2,abc)
        Y= np.fft.fft(x2,LS)/LX
        for k in range(0,LX):
            for l in range(0,LX):
                Bsp[k,l]= Y[k]*Y[l]*np.conjugate(Y[k+l])
                de1[k,l]=abs(Y[k]*Y[l])**2
                de2[k,l]=abs(Y[k+l])**2
        Bs=np.append(Bs,Bsp)
        D1=np.append(D1,de1)
        D2=np.append(D2,de2)
    
    D1= D1.reshape(k1,LX,LX)
    D2 = D2.reshape(k1,LX,LX)
    Bs = Bs.reshape(k1,LX,LX)
    D1m = D1[0]

    D1m = D1[0]
    D2m = D2[0]
    Bm = Bs[0]
    sz2= Bs.shape[0]
    for k in range(0,sz2):
        D1m=D1m+D1[k]
        D2m=D2m+D2[k]
        Bm = Bm+Bs[k]
    D1 = D1m/k
    D2 = D2m/k
    Bm = Bm/k


    bic2 = np.zeros((LX*2,LX*2))
    for k in range(0,LX):
        for l in range(0,LX):
            bic2[k+LX,l+LX] = abs(Bm[k,l])**2 / (D1[k,l]*D2[k,l])

    bican = np.zeros((LX*2,LX*2))
    for k in range(0,LX*2):
        for l in range(0,LX*2):
            if (k<=LX) and (l<=LX):
                bican[k,l] = bic2[(2*LX-k-1),(2*LX-l-1)]
            elif (k>LX) and (l<=LX):
                bican[k,l]=bic2[k,2*LX-l-1]

            elif (k<=LX) and (l>LX):
                bican[k,l]=bic2[2*LX-k-1,l]
            
            elif (k<LX) and (l>bic2[k,l]):
                bican[k,l] = bic2[k,l]

    

    sz=bican.shape

    waxis = np.linspace(0,0.5,sz[1]+1)
    waxis = waxis[:-2]
    limit =waxis[-1]*(2/3)

    k = 0
    while k<len(waxis):
        if waxis[k] > limit :
            limit = k-1
            k=float('inf')
        else :
            k = k+1

    for f1 in range(0,limit):
        for f2 in range(0,len(waxis)):
            if f2 == 0 or f2>= f1:
                bican[f1,f2] = 0
    
    bicpermed = np.array([])
    cont = 0
    for f1 in range(0,len(waxis)):
        for f2 in range(0, min([f1, -2*f1+len(Y)])):
            bicp =  bican[f1,f2]
            cont = cont +1
            bicpermed=np.append(bicpermed,bicp)
    
    bic2m = np.mean(bicpermed)
    bic2v=np.std(bicpermed)
    bic2max = np.max(bicpermed)

    bicant = np.transpose(bican)
    bicant.shape
    c,rows = np.where(bicant[:]==np.max(bicant))

    f1 = rows[0]
    f2 = rows[1]
    fr1 = waxis[f1]
    fr2=waxis[f2]


    #NILAI KETIDAKLINEARAN
    NLI = abs(bic2max-(bic2m+(2*bic2v)))

    #Mengitung NGI
    al = 0.01
    chi = 9.2
    chi2 = chi/(2*LX)

    xi,xii = np.where(bican[:]>chi2)
    b=xi+xii
    n=len(xi)
    biccrit = (1/(4*LX*n))*((chi+np.sqrt((4*n)-1))**2)
    biccritt = chi/(2*LX*k1)
    NGI = bic2m-biccrit
    c=np.sum(bican[xi,xii])

    return NLI, NGI

In [13]:
bicoherence(PV1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: ComplexWarning: Casting complex values to real discards the imaginary part


(0.08377253630332959, 0.3936647117354423)

In [14]:
bicoherence(PV2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: ComplexWarning: Casting complex values to real discards the imaginary part


(0.005883413012135463, 0.35129914428234005)

In [15]:
bicoherence(TC1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: ComplexWarning: Casting complex values to real discards the imaginary part


(0.10712241742369077, 0.40921793634012227)

In [16]:
bicoherence(PC5)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: ComplexWarning: Casting complex values to real discards the imaginary part


(0.12328656210970523, 0.4236711961952824)

In [17]:
bicoherence(PC2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: ComplexWarning: Casting complex values to real discards the imaginary part


(0.08569074250254916, 0.4070447391925892)

In [18]:
bicoherence(PC1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: ComplexWarning: Casting complex values to real discards the imaginary part


(0.09508114090418418, 0.40799639360635115)